In [ ]:
#!/usr/bin/env python
# coding: utf-8
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *

import pipeline_helper, importlib, design_optimization_analysis
with so(): importlib.reload(pipeline_helper)
with so(): importlib.reload(design_optimization_analysis)

import pickle, gzip

from configuration import *

import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 300
OPTS.verboseNonPosDef = False

name = 'squidward_highres'
input_path = '../../data/{}.json.gz'.format(name)

io, input_data, target_mesh, curr_um, thickness, target_height_multiplier = parse_input(input_path, handleBoundary=False, handlePivots = True)
rod_colors = get_color_field(curr_um, input_data)

def get_squidward_view(umbrella):
    import mesh
    view = linkage_vis.LinkageViewerWithSurface(umbrella, target_mesh, width=1024, height=600)
    set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
    view.averagedMaterialFrames = True
    view.setCameraParams(((-16.99669868547026, -1.6155998689650803, 4.863083631031875),
     (0.2619754129859161, 0.1245718618523399, 0.9570009060735004),
     (0.0, 0.0, 0.0)))
    view.showScalarField(rod_colors)
    return view

In [ ]:
curr_um = pickle.load(gzip.open("squidward_data/optimized_single_height_rest_state.pkl.gz", 'r'))

In [ ]:
use_pin = False

driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
view = get_squidward_view(curr_um)
view.show()

In [ ]:
break_input_angle_symmetry(curr_um)

In [ ]:
insert_randomness(curr_um, zPerturbationEpsilon = 1e-2)

In [ ]:
def eqm_callback(prob, i):
    if (i % 5 == 0):
        view.showScalarField(rod_colors)

In [ ]:
configure_umbrella_pre_deployment(curr_um, thickness, target_height_multiplier)

In [ ]:
curr_um.attractionWeight = 1e-6

In [ ]:
results = staged_deployment(curr_um, np.logspace(-4, 0, 5), eqm_callback, OPTS, fixedVars, elasticEnergyIncreaseFactorLimit = 1.5)

In [ ]:
from IPython.display import Image
Image("squidward_data/symmetric.png", width = 1000)

## Encoding Mean Curvature

In [ ]:
curr_um = pickle.load(gzip.open("squidward_data/optimized_rest_state.pkl.gz", 'r'))

In [ ]:
from IPython.display import Image
Image("squidward_data/asymmetric.png", width = 1000)

In [ ]:
use_pin = False

driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 300
OPTS.verboseNonPosDef = False

rod_colors = get_color_field(curr_um, input_data)

In [ ]:
asymmetric_view = get_squidward_view(curr_um)
asymmetric_view.show()

In [ ]:
def eqm_callback(prob, i):
    if (i % 2 == 0):
        asymmetric_view.showScalarField(rod_colors)

In [ ]:
configure_umbrella_pre_deployment(curr_um, thickness, target_height_multiplier)

In [ ]:
curr_um.attractionWeight = 1e-6

In [ ]:
results = staged_deployment(curr_um, np.logspace(-4, 0, 5), eqm_callback, OPTS, fixedVars, elasticEnergyIncreaseFactorLimit = 1.5)